In [16]:
# Library imports

from neo4j import GraphDatabase
import requests
import os
import pandas as pd

In [17]:
#read in data file
psc = pd.read_json('file.txt', lines=True)
psc.head()

,company_number,data
0,09145694,{'address': {'address_line_1': 'St. Andrews Ro...
1,08581893,"{'address': {'address_line_1': 'High Street', ..."
2,08581893,"{'address': {'address_line_1': 'High Street', ..."
3,01605766,"{'address': {'address_line_1': 'Wates House', ..."
4,10259080,{'address': {'address_line_1': 'College Avenue...


In [60]:
# create new columns with name, country, etc
new_cols = ['name', 'etag', 'address']

for col in new_cols:
    psc[col] = psc['data'].apply(lambda x:x[col])

psc['country'] = psc['address'].apply(lambda x: x['country'] if 'country' in x.keys() else 'XXX')
psc.head()

,company_number,data,name,etag,address,country
0,09145694,{'address': {'address_line_1': 'St. Andrews Ro...,Mrs Nga Thanh Wildman,3b8caf795c03af63921e381f7bb8300a51ebb73d,"{'address_line_1': 'St. Andrews Road', 'countr...",England
1,08581893,"{'address': {'address_line_1': 'High Street', ...",Mr Stephen Robert Charles Davies,45f9c9e5494b574eb52abc3990a49bd96fe09df3,"{'address_line_1': 'High Street', 'address_lin...",England
2,08581893,"{'address': {'address_line_1': 'High Street', ...",Mr Quentin Colin Maxwell Solt,d55168c49f85ab1ef38a12ed76238d68f79f5a01,"{'address_line_1': 'High Street', 'address_lin...",England
3,01605766,"{'address': {'address_line_1': 'Wates House', ...",Mr Jeremy Wyckham Wright,198677abde6618a2669e65654da57252bbd6b941,"{'address_line_1': 'Wates House', 'address_lin...",XXX
4,10259080,{'address': {'address_line_1': 'College Avenue...,Dr Muhammad Fayaz Khan,231a5a1071ef608f60a79a0fce2c3e21e29f00b9,"{'address_line_1': 'College Avenue', 'country'...",United Kingdom


In [61]:
# eliminate all the rows with dates of birth - this keeps only the companies
indices = []
for i in range(psc.shape[0]):
    try:
        sth = psc.data.iloc[i]['date_of_birth']
    except KeyError:
        indices.append(i)

companies = psc.iloc[indices]
companies.head()

,company_number,data,name,etag,address,country
36,NI061010,"{'address': {'address_line_1': 'Coast Road', '...",Brian Kelly,a52c3e73eacded61e18e9a0792d06c9b02564bcf,"{'address_line_1': 'Coast Road', 'address_line...",Northern Ireland
39,03072883,{'address': {'address_line_1': 'Warwick Techno...,The Midcounties Co-Operative Properties Limited,e6870db4a5c81c15852b7eec668afcff47f5f9d2,"{'address_line_1': 'Warwick Technology Park', ...",England
56,01935622,{'address': {'address_line_1': 'Warwick Techno...,Tilley & Daniells Limited,d1687225589d31666bb54884630c2a31c555ff3b,"{'address_line_1': 'Warwick Technology Park', ...",England
71,08593845,{'address': {'address_line_1': 'Telok Ayer Str...,David Wood & Associates Pte Ltd,0603ea29438304185b66ff68b92d8e1e0e93736d,"{'address_line_1': 'Telok Ayer Street', 'addre...",Singapore
78,02225091,{'address': {'address_line_1': 'St. Margarets ...,Rpow Uk Limited,208b8fc3d8caca327e638224d2e348a619186fd1,"{'address_line_1': 'St. Margarets Street', 'co...",England


In [64]:
dbase_uri = "bolt://localhost:7687"
authenticator = ("neo4j", "Passw0rd")

def create_connection(uri, authenticator):    
    data_base_connection = GraphDatabase.driver(uri = uri , auth=authenticator)
    return data_base_connection.session() 

In [66]:
session = create_connection(dbase_uri, authenticator)
def delete_connections(session):  
    query = "match (n) detach delete n"
    session.run(query)

delete_connections(session)

In [67]:
company_commands = []
how_many = 50
for i in range(how_many):
    neo4j_create_statement = "MERGE (:Company {{company_number: '{}'}})".format(str(companies.company_number.iloc[i]))
    company_commands.append(neo4j_create_statement)

person_commands = []
for i in range(how_many):
    neo4j_create_statement = 'MERGE (:Person {{name: "{}", psc_of: ["{}"]}})'.format(companies.iloc[i]['name'], 
                                                                                      str(companies.company_number.iloc[i]))

    person_commands.append(neo4j_create_statement)
    
def create_company(company_creation_commands):   
    for command in company_creation_commands:
        session.run(command)

def create_person(person_creation_commands):
    for command in person_creation_commands:
        session.run(command)

create_company(company_commands)
create_person(person_commands)

In [68]:
create_relationships = "match (p:Person) match(c:Company) WHERE c.company_number IN p.psc_of MERGE (p)-[:psc_of] -> (c)"
session.run(create_relationships)

### Next steps: Evaluate complex company structures
1. Which companies have multiple owners? Maybe write a function that takes in a threshold, and returns all the companies with a number of owners greater than that threshold

2. Find companies whose owners are not in the UK. To do this, create a cnew column with the `country` of the owner, and then we can filter out those that are not in the UK.